In [1]:
import os
import pandas as pd
import numpy as np
import json
from sklearn.utils import shuffle

In [2]:
read_dir='E:/文档/数据集/Data Sets used in Computing Education/junyi/'
save_dir='../data/junyi/'

log=pd.read_csv(read_dir+'junyi_ProblemLog_original.csv')

prob_count_limit=15 #每个学生做过的习题下限

In [33]:
data=log.loc[:,['user_id','exercise','correct']].drop_duplicates(subset=['user_id','exercise'],
        keep='first').dropna(axis=0,how='any')
data['correct']=data['correct']*1

In [34]:
# 统计每个学生做了多少道题
problem_counter=data.groupby(by='user_id').count()

filtered_stu_boundary=prob_count_limit #学生做的题超过15道才算数

filtered_stu_id=problem_counter[problem_counter['exercise']>filtered_stu_boundary].index.to_numpy()

print('有效的学生数：',len(filtered_stu_id))

record=data.set_index('user_id').loc[filtered_stu_id,:].reset_index()
record.columns=['user_id','item_id','score']

有效的学生数： 36591


In [35]:
item_conc_data=record.loc[:,['item_id','item_id']].dropna(axis=0,how='any').drop_duplicates()
item_conc_data.columns=['item_id','knowledge_code']
item_conc_data=item_conc_data.set_index('item_id')
item_conc_data=item_conc_data[~item_conc_data.index.duplicated()]
item_conc_data=item_conc_data.reset_index()

In [36]:
item_unique=np.unique(record['item_id'])
know_unique=np.unique(item_conc_data['knowledge_code'])
stu_unique=np.unique(record['user_id'])

In [37]:
stu_old_new=dict(zip(stu_unique,range(1,len(stu_unique)+1)))
item_old_new=dict(zip(item_unique,range(1,len(item_unique)+1)))
know_old_new=dict(zip(know_unique,range(1,len(know_unique)+1)))

In [38]:
with open(save_dir+'dict_knowledge_code.json','w') as f:
    json.dump(know_old_new,f)

In [39]:
record['user_id']=record['user_id'].map(stu_old_new)
record['item_id']=record['item_id'].map(item_old_new)

item_conc_data['item_id']=item_conc_data['item_id'].map(item_old_new)
item_conc_data['knowledge_code']=item_conc_data['knowledge_code'].map(know_old_new)

print('Done')

Done


In [40]:
item_conc_data=item_conc_data.set_index('item_id')

item_df=pd.DataFrame(columns=['item_id','knowledge_code'],index=range(1,len(item_unique)+1))
for item in range(1,len(item_unique)+1):
    item_df.loc[item,'item_id']=item
    item_df.loc[item,'knowledge_code']=np.array(item_conc_data.loc[item,['knowledge_code']]).reshape(-1).tolist()

In [41]:
item_df.to_csv(save_dir+'item.csv',index=False)
record.to_csv(save_dir+'record.csv',index=False)

In [42]:
print('学习者数：',len(stu_unique))
print('习题数：',len(item_unique))
print('知识点数：',len(know_unique))
print('记录数：',len(record))

学习者数： 36591
习题数： 721
知识点数： 721
记录数： 1550016


In [43]:
relation_data_dir='E:/文档/数据集/Data Sets used in Computing Education/junyi/'
prob_data=pd.read_csv(relation_data_dir+'junyi_Exercise_table.csv')

prerequisites=prob_data.loc[:,['name','prerequisites']]
prerequisites['name']=prerequisites['name'].map(know_old_new)
prerequisites['prerequisites']=prerequisites['prerequisites'].map(know_old_new)

prerequisites=prerequisites.dropna(how='any').astype('int')
prerequisites.columns=['Exercise_A','Exercise_B']

In [44]:
relation_test=pd.read_csv(relation_data_dir+'relationship_annotation_testing.csv')
relation_train=pd.read_csv(relation_data_dir+'relationship_annotation_training.csv')

relation_data=pd.concat([relation_test,relation_train]).loc[:,['Exercise_A','Exercise_B','Similarity_avg']]
relation_data=relation_data[relation_data['Similarity_avg']>=5].loc[:,['Exercise_A','Exercise_B']]
relation_data['Exercise_A']=relation_data['Exercise_A'].map(know_old_new)
relation_data['Exercise_B']=relation_data['Exercise_B'].map(know_old_new)
relation_data=relation_data.dropna(how='any').astype('int')

# 相似关系是相互的
relation_data_2=pd.DataFrame(columns=['Exercise_A','Exercise_B']) 
relation_data_2['Exercise_A']=relation_data['Exercise_B']
relation_data_2['Exercise_B']=relation_data['Exercise_A']

relationship=pd.concat([prerequisites,relation_data,relation_data_2]).drop_duplicates(keep='first')
relationship.columns=['knowledge_code','parent']

In [45]:
relationship.to_csv(save_dir+'concept_relationship.csv',index=False)